In [3]:
using LinearAlgebra, BenchmarkTools

In [4]:
#u_{11}=2,\, u_{12}=3,\, l_{21}=1,\, l_{22}=-1 
u11=2
u12=4
l21=0.0/u11
u22=3.0 - (l21*u12) 
L, U = [1 0; l21 1], [u11 u12;0 u22]

([1.0 0.0; 0.0 1.0], [2.0 4.0; 0.0 3.0])

In [286]:
function PLU(A::Matrix{T}; eptols = 1.0e-10) where T<:Real
    M, N = size(A)
    @assert M == N

    L, P, U = one(A), one(A), zero(A)
    B = copy(A)
    
    for i in 1:(M-1)
        p = argmax(abs.(B[i:end, i])) + i -1
        
        if abs(B[p, i]) < eptols
            error("Singularity error")    
        end

        P[i,:], P[p, :] = P[p, :], P[i, :]
        B[i,:], B[p, :] = B[p, :], B[i, :]
        
        if i>1
            L[i,1:i-1], L[p, 1:i-1] = L[p, 1:i-1], L[i, 1:i-1]
        end
        
        U[i, i] = B[i, i]
        U[i, (i+1):end] = B[i, (i+1):end]
        L[(i+1):end, i] = B[(i+1):end, i] / B[i, i]
        B[(i+1):end, (i+1):end] = B[(i+1):end, (i+1):end] - (L[(i+1):end, i:i] * U[i:i, (i+1):end])
    end
    U[M, M] = B[M, M]
    return P, L, U
end

function plufact(A)
    n = size(A,1)
    L = zeros(n,n)
    U = zeros(n,n)
    p = fill(0,n)
    Aₖ = float(copy(A))

    # Reduction by outer products
    for k in 1:n-1
        p[k] = argmax(abs.(Aₖ[:,k]))
        U[k,:] = Aₖ[p[k],:]
        L[:,k] = Aₖ[:,k]/U[k,k]
        Aₖ -= L[:,k]*U[k,:]'
    end
    p[n] = argmax(abs.(Aₖ[:,n]))
    U[n,n] = Aₖ[p[n],n]
    L[:,n] = Aₖ[:,n]/U[n,n]
    return p,  LowerTriangular(L[p,:]), U
end

plufact (generic function with 1 method)

In [293]:
n=10
A = float.(rand(1:100, (n, n)))
P, L, U = PLU(A)

([1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1.0 0.0 … 0.0 0.0; 0.04 1.0 … 0.0 0.0; … ; 0.52 -0.0993150684931507 … 1.0 0.0; 0.32 0.8082191780821918 … -0.7568729799322469 1.0], [100.0 23.0 … 27.0 33.0; 0.0 70.08 … 25.92 86.68; … ; 0.0 0.0 … 54.29836021589808 -6.149290364661873; 0.0 0.0 … 0.0 13.52412409106661])

In [294]:
L

10×10 Matrix{Float64}:
 1.0    0.0         0.0        …   0.0       0.0        0.0       0.0
 0.04   1.0         0.0            0.0       0.0        0.0       0.0
 0.59   0.0204053   1.0            0.0       0.0        0.0       0.0
 0.57   0.854595    0.658493       0.0       0.0        0.0       0.0
 0.73   0.659389    0.533538       0.0       0.0        0.0       0.0
 0.81   0.561787    0.431572   …   0.0       0.0        0.0       0.0
 0.06   0.865011    0.671655       1.0       0.0        0.0       0.0
 0.24   0.163813   -0.0375045      0.720983  1.0        0.0       0.0
 0.52  -0.0993151  -0.432474      -0.177569  0.42989    1.0       0.0
 0.32   0.808219    0.0227191      0.712318  0.972258  -0.756873  1.0

In [295]:
P*A-L*U

10×10 Matrix{Float64}:
 0.0          0.0   0.0           0.0          …   0.0           0.0
 0.0          0.0   0.0           0.0              0.0           0.0
 0.0          0.0   0.0           0.0              0.0           0.0
 7.10543e-15  0.0   1.42109e-14   1.42109e-14      0.0           0.0
 0.0          0.0   0.0           1.42109e-14     -7.10543e-15   0.0
 0.0          0.0   0.0           0.0          …   0.0          -1.42109e-14
 0.0          0.0   0.0           1.06581e-14      0.0           0.0
 0.0          0.0   0.0           0.0              1.42109e-14   0.0
 0.0          0.0   0.0           0.0              0.0           1.77636e-14
 0.0          0.0  -1.42109e-14  -3.55271e-15      0.0           0.0

In [276]:
p1, L1, U1 = plufact(A)

([1, 5, 2, 4, 3], [1.0 0.0 … 0.0 0.0; 0.125 1.0 … 0.0 0.0; … ; 0.625 -0.28695652173913044 … 1.0 0.0; 0.125 0.9304347826086956 … -0.7107517106081629 1.0], [16.0 13.0 … 4.0 15.0; 0.0 14.375 … 11.5 9.125; … ; 0.0 0.0 … 5.2735301539237955 5.334595003785012; 0.0 0.0 … 0.0 -1.4317431456050533])

In [279]:
L-L1

5×5 Matrix{Float64}:
 0.0   0.0   0.0  0.0   0.0
 0.0  -1.0   0.0  0.0   1.0
 0.0   0.0  -1.0  0.0   0.0
 0.0   0.0   0.0  0.0   0.0
 0.0   0.0   0.0  0.0  -1.0

In [278]:
U1

5×5 Matrix{Float64}:
 16.0  13.0     1.0      4.0      15.0
  0.0  14.375  10.875   11.5       9.125
  0.0   0.0    17.2304  19.6      -9.40435
  0.0   0.0     0.0      5.27353   5.3346
  0.0   0.0     0.0      0.0      -1.43174

In [273]:
A[p1,:] - L1*U1

5×5 Matrix{Float64}:
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0  -1.77636e-15  0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0

In [237]:
P

5×5 Matrix{Float64}:
 0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0

In [147]:
function LU(A::Matrix{T}; eptols = 1.0e-10) where T<:Real
    M, N = size(A)
    @assert M == N

    L, U = one(A), one(A), zero(A)
    B = copy(A)
    
    for i in 1:(M-1)
        p = argmax(abs.(B[i:end, i])) + i -1
        
        if abs(B[p, i]) < eptols
            error("Singularity error")    
        end
        
        U[i, :] = B[i, :]
        U[i, :] .= B[i, :]
        L[(i+1):end, i] .= B[(i+1):end, i] ./ U[i, i]
        B[(i+1):end, (i+1):end] .= B[(i+1):end, (i+1):end] - (L[(i+1):end, i:i] * U[i:i, (i+1):end])
    end
    U[N,N] = B[N,N]
    L[:,N] = B[:,N]/U[N,N]
    return P, L, U
end

LU (generic function with 1 method)

In [148]:
function LU2(A::Matrix{T}) where T<:AbstractFloat
    U = copy(A)
    M, N = size(U)[1:2]
    @assert M == N
    L = one(A)
    for i in 2:N
        L[i, 1] = A[i, 1]/A[1, 1]
        U[i, 1] = 0.0 
    end
    for i = 2:N 
        for j = 2:N 
            fc = zero(T)
            for k = 1:i-1
                fc += L[i, k]*U[k, j]
            end
            if i>j
                L[i, j] = (A[i, j] - fc)/U[j, j]
                U[i, j] = zero(T)
            else 
                U[i, j] = (A[i, j] - fc) 
            end
        end
    end
    return L, U
end

function LU1(A::Matrix{T}) where T<:AbstractFloat
    U = copy(A)
    M, N = size(U)[1:2]
    @assert M == N
    L = one(A)x

    for i = 2:N 
        for j = 2:N 
            fc = zero(T)
            for k = 1:i-1
                fc += L[i, k]*U[k, j]
            end
            if i>j
                L[i, j] = (A[i, j] - fc)/U[j, j]  
                U[i, j] = zero(T)
            else 
                U[i, j] = (A[i, j] - fc) 
            end
        end
    end
    return L, U
end


LU1 (generic function with 1 method)

In [80]:
Ac = Float64.([3 4 2 5; 5 3 4 -2; -2 -3 1 0; -2 -3 1 -2])
b = Float64.([3, 2, 1, -2])
l1, u1 = LU(Ac)

([1.0 0.0 0.0 0.0; 1.6666666666666667 1.0 0.0 0.0; -0.6666666666666666 0.09090909090909094 1.0 0.0; -0.6666666666666666 0.09090909090909094 1.0 1.0], [3.0 4.0 2.0 5.0; 0.0 -3.666666666666667 0.6666666666666665 -10.333333333333334; 0.0 0.0 2.2727272727272725 4.2727272727272725; 0.0 0.0 0.0 -2.0])

In [90]:
A=reshape(collect(1:16), (4, 4))

4×4 Matrix{Int64}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

In [91]:
A[1, :], A[3, :] = A[3, :], A[:, 1]

([3, 7, 11, 15], [1, 2, 3, 4])

In [114]:
@show A

A = [0.7876136060778486 0.24140264640800413 0.9883308798271858 5.0 0.8489136798606083; 0.6136753245309301 0.45169857158927706 0.28445889363912746 0.7461957362532627 0.25910228196949836; 0.6410401021823309 4.0 0.5858200156319991 0.5398806153414019 0.6846274278328139; 0.7079054600652507 0.7546581498147679 0.031131735623354806 0.3835467507327587 0.9438646751260223; 0.3492444116453274 0.08153239911663812 0.7018301516069582 0.31184691724967606 0.795967203835045]


5×5 Matrix{Float64}:
 0.787614  0.241403   0.988331   5.0       0.848914
 0.613675  0.451699   0.284459   0.746196  0.259102
 0.64104   4.0        0.58582    0.539881  0.684627
 0.707905  0.754658   0.0311317  0.383547  0.943865
 0.349244  0.0815324  0.70183    0.311847  0.795967

In [117]:
@show A

A = [0.7876136060778486 0.24140264640800413 0.9883308798271858 5.0 0.8489136798606083; 0.6136753245309301 0.45169857158927706 0.28445889363912746 0.7461957362532627 0.25910228196949836; 0.6410401021823309 4.0 0.5858200156319991 0.5398806153414019 0.6846274278328139; 0.7079054600652507 0.7546581498147679 0.031131735623354806 0.3835467507327587 0.9438646751260223; 0.3492444116453274 0.08153239911663812 0.7018301516069582 0.31184691724967606 0.795967203835045]


5×5 Matrix{Float64}:
 0.787614  0.241403   0.988331   5.0       0.848914
 0.613675  0.451699   0.284459   0.746196  0.259102
 0.64104   4.0        0.58582    0.539881  0.684627
 0.707905  0.754658   0.0311317  0.383547  0.943865
 0.349244  0.0815324  0.70183    0.311847  0.795967